In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import os
import subprocess

#import colors
import time
import datetime
#from rfc3339 import rfc3339

In [2]:
def init_splinter():
    '''
        init_splinter() initializes and returns an automated Chrome web browser for us to use for
    scraping.
    '''
    print('Initizalizing Splinter Browser...\n')
    time.sleep(2)
    ############################################
    if (os.name == "posix"):    
        executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    elif (os.name == "nt"):
        executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    ############################################
    return browser


def simmer_soup():
    '''
        simmer_soup() receives the Splinter browser and returns the current page's parsed html as
    "Soup".
    '''
    html = browser.html
    soup = bs(html, 'html.parser')
    ############################################
    return html, soup

In [3]:
# Init the browser
browser = init_splinter()
browser.driver.minimize_window()

Initizalizing Splinter Browser...



In [4]:
# Download the csv to ~/Downloads as "API_SH.H2O.SMDW.ZS_DS2_en_csv_v2_10581721.zip"
url = "http://api.worldbank.org/v2/en/indicator/SH.H2O.SMDW.ZS?downloadformat=csv"
browser.visit(url)

In [5]:
# return the correct slash for the os
if (os.name == "posix"):
    slash = "/"
elif (os.name == "nt"):
    slash = "\\"

In [6]:
# create a var of the user's downloads dir
data_dir = str(os.environ['HOME'] + slash + "Downloads" + slash)
print(data_dir)

/Users/nicolespaar/Downloads/


In [7]:
# create a var of the downloaded archive of .csvs
data_file = "API_SH.H2O.SMDW.ZS_DS2_en_csv_v2_10581721.zip"
data_loc = (data_dir + data_file)
print(data_loc)

/Users/nicolespaar/Downloads/API_SH.H2O.SMDW.ZS_DS2_en_csv_v2_10581721.zip


In [9]:
# extract the .csvs from the archive into the same downloads directory
import zipfile
zf = zipfile.ZipFile(data_loc, 'r')
zf.extractall(data_dir)

In [10]:
csv1_name = "API_SH.H2O.SMDW.ZS_DS2_en_csv_v2_10581721.csv"
csv1 = (data_dir + csv1_name)
print(csv1)

/Users/nicolespaar/Downloads/API_SH.H2O.SMDW.ZS_DS2_en_csv_v2_10581721.csv


In [11]:
csv2_name = "Metadata_Country_API_SH.H2O.SMDW.ZS_DS2_en_csv_v2_10581721.csv"
csv2 = (data_dir + csv2_name)
print(csv2)

/Users/nicolespaar/Downloads/Metadata_Country_API_SH.H2O.SMDW.ZS_DS2_en_csv_v2_10581721.csv


In [24]:
csv1_DF = pd.read_csv(csv1, header=2)
csv1_DF_clean = csv1_DF[["Country Name", "Country Code", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015"]].copy()
csv1_DF_clean = csv1_DF_clean.dropna(axis=0)
csv1_DF_clean = csv1_DF_clean.drop([62, 63, 229, 215, 151, 71, 132, 93, 103, 257], axis=0)
### output to csv is temporary for group work to continue before sql integration
#csv1_DF_clean.to_csv("access.csv")
###
csv1_DF_clean.head()

,Country Name,Country Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
3,Albania,ALB,43.235267,45.108835,46.966948,48.821304,50.671535,52.517182,54.357921,56.193358,58.023103,59.846676,61.663829,63.474112,65.277205,67.072394,68.859504,68.869979
7,Argentina,ARG,97.694744,97.743560,97.791188,97.837488,97.882309,97.996731,98.109610,98.220977,98.269485,98.306944,98.343078,98.377874,98.411365,98.409375,98.532065,98.527415
8,Armenia,ARM,26.701348,26.747758,31.052874,35.375587,39.714745,43.889582,46.715767,49.531490,52.330072,55.094412,57.825350,60.499950,60.424483,60.355636,60.617571,60.581446
12,Austria,AUT,97.830153,97.887471,97.944788,98.002106,98.059423,98.116740,98.174058,98.231375,98.288693,98.346010,98.403327,98.460645,98.517962,98.575280,98.632597,98.689914
13,Azerbaijan,AZE,51.422195,51.797960,52.171309,52.542584,54.664516,56.808342,58.974001,61.161429,63.370085,65.599908,67.852341,69.785707,70.214670,70.646412,71.080776,71.517164


In [23]:
csv2_DF = pd.read_csv(csv2)
csv2_DF_clean = csv2_DF.drop(["Unnamed: 5"], axis=1).dropna(axis=0)
### output to csv is temporary for group work to continue before sql integration
#csv2_DF_clean.to_csv("income_groups.csv")
###
csv2_DF_clean.head()

,Country Code,Region,IncomeGroup,SpecialNotes,TableName
0,ABW,Latin America & Caribbean,High income,Central Bureau of Statistics and Central Bank ...,Aruba
1,AFG,South Asia,Low income,Central Statistics Organization; World Bank st...,Afghanistan
2,AGO,Sub-Saharan Africa,Lower middle income,IMF ; Source of population estimates: UN Popul...,Angola
3,ALB,Europe & Central Asia,Upper middle income,Albanian Institute of Statistics ; Source of p...,Albania
4,AND,Europe & Central Asia,High income,"Government of Andorra, Department of Statistic...",Andorra


In [36]:
DF = pd.merge(csv1_DF_clean, csv2_DF_clean, on="Country Code", how="left")
DF = DF.drop(["Region", "SpecialNotes", "TableName"], axis=1).copy()
DF.to_csv("water_data.csv")
DF.head()

,Country Name,Country Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,IncomeGroup
0,Albania,ALB,43.235267,45.108835,46.966948,48.821304,50.671535,52.517182,54.357921,56.193358,58.023103,59.846676,61.663829,63.474112,65.277205,67.072394,68.859504,68.869979,Upper middle income
1,Argentina,ARG,97.694744,97.743560,97.791188,97.837488,97.882309,97.996731,98.109610,98.220977,98.269485,98.306944,98.343078,98.377874,98.411365,98.409375,98.532065,98.527415,High income
2,Armenia,ARM,26.701348,26.747758,31.052874,35.375587,39.714745,43.889582,46.715767,49.531490,52.330072,55.094412,57.825350,60.499950,60.424483,60.355636,60.617571,60.581446,Upper middle income
3,Austria,AUT,97.830153,97.887471,97.944788,98.002106,98.059423,98.116740,98.174058,98.231375,98.288693,98.346010,98.403327,98.460645,98.517962,98.575280,98.632597,98.689914,High income
4,Azerbaijan,AZE,51.422195,51.797960,52.171309,52.542584,54.664516,56.808342,58.974001,61.161429,63.370085,65.599908,67.852341,69.785707,70.214670,70.646412,71.080776,71.517164,Upper middle income


In [37]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///water_data.sqlite', echo=True)
DF.to_sql("water_data", con=engine, if_exists="replace")

2019-06-24 18:46:51,559 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-06-24 18:46:51,560 INFO sqlalchemy.engine.base.Engine ()
2019-06-24 18:46:51,561 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-06-24 18:46:51,562 INFO sqlalchemy.engine.base.Engine ()
2019-06-24 18:46:51,563 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("water_data")
2019-06-24 18:46:51,564 INFO sqlalchemy.engine.base.Engine ()
2019-06-24 18:46:51,570 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE water_data (
	"index" BIGINT, 
	"Country Name" TEXT, 
	"Country Code" TEXT, 
	"2000" FLOAT, 
	"2001" FLOAT, 
	"2002" FLOAT, 
	"2003" FLOAT, 
	"2004" FLOAT, 
	"2005" FLOAT, 
	"2006" FLOAT, 
	"2007" FLOAT, 
	"2008" FLOAT, 
	"2009" FLOAT, 
	"2010" FLOAT, 
	"2011" FLOAT, 
	"2012" FLOAT, 
	"2013" FLOAT, 
	"2014" FLOAT, 
	"2015" FLOAT, 
	"IncomeGroup" TEXT
)


2019-06-24 18:46:51,571 INFO sqlalchemy.engine.base.Eng

In [38]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func, inspect


In [40]:
#Base = automap_base()
#Base.prepare(access_engine, reflect=True)
#Base.classes.keys()

2019-06-24 18:47:11,153 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-06-24 18:47:11,154 INFO sqlalchemy.engine.base.Engine ()
2019-06-24 18:47:11,156 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("access")
2019-06-24 18:47:11,157 INFO sqlalchemy.engine.base.Engine ()
2019-06-24 18:47:11,160 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'access' AND type = 'table'
2019-06-24 18:47:11,160 INFO sqlalchemy.engine.base.Engine ()
2019-06-24 18:47:11,162 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("access")
2019-06-24 18:47:11,163 INFO sqlalchemy.engine.base.Engine ()
2019-06-24 18:47:11,164 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'access' AND type = 'table'
2019-06-24 18:47:11,165 INFO sqlalchemy.engine.base.Engine ()
201

[]

In [42]:
session = Session(engine)
test = session.execute("select * from water_data")
for item in test:
    print(item)
    print("\n")

2019-06-24 18:47:35,374 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-06-24 18:47:35,375 INFO sqlalchemy.engine.base.Engine select * from water_data
2019-06-24 18:47:35,375 INFO sqlalchemy.engine.base.Engine ()
(0, 'Albania', 'ALB', 43.2352673384537, 45.1088352996924, 46.9669478266825, 48.82130378234779, 50.671535281237404, 52.517182087092294, 54.3579208624669, 56.193358439833794, 58.0231032318432, 59.84667610056021, 61.663828610535404, 63.4741124365256, 65.2772050915469, 67.0723939914111, 68.8595042120187, 68.869979425596, 'Upper middle income')


(1, 'Argentina', 'ARG', 97.69474394501101, 97.74355979258681, 97.79118751006521, 97.83748803177241, 97.8823089722946, 97.9967305456147, 98.1096100264979, 98.2209766875463, 98.2694852624099, 98.30694425349579, 98.3430777803307, 98.3778741314803, 98.4113653593348, 98.409375035814, 98.5320653022513, 98.5274147955714, 'High income')


(2, 'Armenia', 'ARM', 26.7013479268149, 26.747757654028202, 31.0528744424918, 35.3755867781183, 39.71